# Two-Dimensional Aquifer Thermal Energy Storage (ATES) in a single layer with the GroundWater Engergy (GWE) package

In this notebook, we will learn how to:
1. Setup a MODFLOW 6 model for an ATES doublet (a warm well and a cold well).
2. Perform multiple cycles.
3. Compute the thermal recovery efficiency.
4. Investigate the effect of putting the warm and cold wells closer together. 

In [ ]:
# import the necessary packages
import numpy as np # the numpy package
import matplotlib.pyplot as plt # the plotting part of matplotlib
plt.rcParams['figure.figsize'] = (8, 2.5) # set default figure size
import flopy as fp  # import flopy and call it fp
from ipywidgets import interact, FloatSlider, Layout # import some widget functions
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # don't show DeprecationWarning-s in this notebook

## Description of the flow problem
Consider two-dimensional flow with a warm well and a cold well. The warm well inject water while the cold well extract water and vice versa. The model area is a rectangle of 505 m by 305 m. Cells are 5 by 5 m (relatively large so that the model runs quickly), so the model consists of 61 rows and 101 columns. The initial temperature of the aquifer is 12$^\circ$C. Injection temperature of the warm well is 17$^\circ$C and of the cold well 7$^\circ$C. The warm well is located at $(x,y)=(-100,0)$ and the cold well at $(x,y)=(100,0)$ (the origin is at the center of the model). The head is fixed to $h_R$ along the entire model boundary. Flow is considered to be at steady state instantaneously. The model consists of 1 layer to make sure the model runs fast enough for this workshop. 

### Parameters

In [ ]:
# aquifer parameters
k = 35 # hydraulic conductivity, m/d
H = 20 # aquifer thickness, m
theta = 0.3 # porosity, -

# energy parameters
rhow = 1000 # density of water, kg/m^3
rhos = 2640 # density of solids, kg/m^3
kappaw = 0.58 * 86400 # thermal conductivity water, J/(d m C)
kappas = 3 * 86400 # thermal conductivity solids, J/(d m C)
cpw = 4180 # specific heat capacity water, J/(kg C)
cps = 710 # specific heat capacity solids, J/(kg C)

# transport
alphaL = 0.1 # longitudinal dispersivity in horizontal direction, m
alphaT = alphaL / 10 # transverse dispersivity is 10 times smaller than longitudinal, m

# boundary condition
hR = 0 # head at model boundary

# wells
Q = 1200 # well discharge, m^3/d
xwarm = -100 # x-location of warm well, m
ywarm = 0 # y-location of warm well, m
xcold = 100 # x-location of cold well, m
ycold = 0 # y-location of cold well, m

# temperature
Tinit = 12 # initial temperature, C
Twarm = 17 # temperature injected warm water, C
Tcold = 7 # temperature injected cold water, C

# space discretization
delx = 5 # length of cell in x and y direction, m
nlay = 1 # number of layers
nrow = 61 # number of rows
ncol = 101 # number of columns
z = [0, -H] # elevation of top and bottom of aquifer

# time and time discretization
tin = 180 # injection time, d
delt = 1 # time step, d
nstepin = round(tin / delt) # computed number of steps during injection
tout = 180 # extraction time, d
delt = 1 # time step, d
nstepout = round(tout / delt) # computed number of steps during extraction

# model name and workspace
modelname = 'gwe2d' # name of model
gwfname = modelname + 'f' # name of flow model
gwename = modelname + 'e' # name of energy model
modelws = './' + modelname # model workspace to be used (where MODFLOW will store all the files)

## Model
The `atesmodel` takes as input the $(x,y)$ locations of the wells (so we can change them easily later) and the number of cycles. When creating the grid, the $x$ and $y$ coordinates of the lower-left-hand corner of the grid are specified. The row and column number of the cells that contain the wells can then be determined with `flopy` commands.  Both a warm and a cold well are specified with equal but opposite discharge. Constant-head cells are specified along the entire model boundary. Other than that, the function is very similar to the function we used in the first notebook.

In [ ]:
def atesmodel(xwarm, ywarm, xcold, ycold, ncycle):
    # simulation
    sim = fp.mf6.MFSimulation(sim_name=modelname, # name of simulation
                              version='mf6', # version of MODFLOW
                              exe_name='mf6', # path to MODFLOW executable
                              sim_ws=modelws, # path to workspace where all files are stored
                             )

    # time discretization
    tdis = fp.mf6.ModflowTdis(simulation=sim, # add to the simulation called sim (defined above)
                              time_units="DAYS", 
                              nper=2 * ncycle, # number of stress periods 
                              perioddata=ncycle * [[tin, nstepin, 1], # period length, number of steps, timestep multiplier
                                        [tout, nstepout, 1]],
                             )
    
    # groundwater flow model
    gwf = fp.mf6.ModflowGwf(simulation=sim, # add to simulation called sim
                            modelname=gwfname, # name of gwf model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwf_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation called sim
                                 filename=gwf.name + '.ims', # file name to store ims
                                 linear_acceleration="BICGSTAB", # use BIConjuGantGradientSTABalized method
                                )                                                                                                
    # register solver
    sim.register_ims_package(solution_file=gwf_ims, # name of iterative model solver instance
                             model_list=[gwf.name], # list with name of groundwater flow model
                            )   
    
    # discretization
    gwf_dis = fp.mf6.ModflowGwfdis(model=gwf, # add to groundwater flow model called gwf
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delx, 
                                   delc=delx, 
                                   top=z[0], 
                                   botm=z[1:],
                                   xorigin=-(ncol / 2) * delx,
                                   yorigin=-(nrow / 2) * delx,
                                  )
    
    # aquifer properties
    gwf_npf  = fp.mf6.ModflowGwfnpf(model=gwf, 
                                    k=k, # horizontal k value
                                    save_flows=True, # save the flow for all cells
                                   )
        
    # initial condition
    gwf_ic = fp.mf6.ModflowGwfic(model=gwf, 
                                 strt=hR, # initial head used for iterative solution
                                )

    # wells
    iwarm, jwarm = gwf.modelgrid.intersect(xwarm, ywarm)
    icold, jcold = gwf.modelgrid.intersect(xcold, ycold)
    wellin = [[(0, iwarm, jwarm), Q, Twarm], 
              [(0, icold, jcold), -Q, Tcold]] # [(layer, row, col), Q, temperature] during injection
    wellout = [[(0, iwarm, jwarm), -Q, Twarm], 
              [(0, icold, jcold), Q, Tcold]]    
    wel_spd = {}
    for i in range(ncycle):
        wel_spd[2 * i] = wellin
        wel_spd[2 * i + 1] = wellout
    gwf_wel = fp.mf6.ModflowGwfwel(model=gwf, 
                                   stress_period_data=wel_spd, 
                                   auxiliary=['TEMPERATURE'],
                                   pname='WEL1', # package name
                                  )
    
    # constant head
    chd0 = []
    for jcol in range(ncol):
        chd0.append([(0, 0, jcol), hR, Tinit]) # [(layer, row, col), head, temperature]
        chd0.append([(0, nrow - 1, jcol), hR, Tinit])
    for irow in range(1, nrow - 1):
        chd0.append([(0, irow, 0), hR, Tinit])
        chd0.append([(0, irow, ncol - 1), hR, Tinit])   
    chd_spd  = {0: chd0} # stress period data
    gwf_chd = fp.mf6.ModflowGwfchd(model=gwf, 
                                   stress_period_data=chd_spd, 
                                   auxiliary=['TEMPERATURE'],
                                   pname='CHD1', # package name
                                  )
        
    # output control
    oc = fp.mf6.ModflowGwfoc(model=gwf, 
                             saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwfname}.cbc", # file name where all budget output is stored
                             head_filerecord=f"{gwfname}.hds", # file name where all head output is stored
                            )
    
    # groundwater energy model
    gwe = fp.mf6.ModflowGwe(simulation=sim, # add to simulation called sim
                            modelname=gwename, # name of gwf model
                            save_flows=True, # make sure all flows are stored in binary output file
                           )
    
    # iterative model solver
    gwe_ims  = fp.mf6.ModflowIms(simulation=sim, # add to simulation
                                 filename=gwe.name + '.ims', # must be different than file name of gwf model ims
                                 linear_acceleration="BICGSTAB",
                                ) 
    sim.register_ims_package(solution_file=gwe_ims, 
                             model_list=[gwe.name],
                            )
    
    # discretization
    gwe_dis = fp.mf6.ModflowGwedis(model=gwe, # add to gwe model
                                   nlay=nlay, 
                                   nrow=nrow, 
                                   ncol=ncol, 
                                   delr=delx, 
                                   delc=delx, 
                                   top=z[0], 
                                   botm=z[1:], 
                                   xorigin=-(ncol / 2) * delx,
                                   yorigin=-(nrow / 2) * delx,
                                  )
    
    # initial condition
    gwe_ic = fp.mf6.ModflowGweic(model=gwe, 
                                 strt=Tinit, # initial temperature
                                ) 
    
    # advection
    adv = fp.mf6.ModflowGweadv(model=gwe,  
                               #scheme="upstream", # use the upstream method
                               scheme="tvd", # use the upstream method
                               pname='ADV1',
                              )
    
    # energy storage package
    gwe_sto = fp.mf6.ModflowGweest(model=gwe, 
                                   porosity=theta, # porosity
                                   heat_capacity_water=cpw,
                                   density_water=rhow,
                                   heat_capacity_solid=cps,
                                   density_solid=rhos,
                                   pname="EST",
                                   save_flows=True,
                                  )
    
    # conduction
    dsp = fp.mf6.ModflowGwecnd(model=gwe, 
                               xt3d_off=True,
                               alh=alphaL, # longitudinal dispersivity
                               ath1=alphaT, # transverse dispersivity
                               ktw=kappaw,
                               kts=kappas,
                               pname='CND', 
                              )
    
    # source sink mixing
    sourcelist = [("WEL1", "AUX", "TEMPERATURE"), ("CHD1", "AUX", "TEMPERATURE")] # list of (pname, 'AUX', 'TEMPERATURE')
    ssm = fp.mf6.ModflowGwessm(model=gwe, 
                               sources=sourcelist, 
                               save_flows=True,
                               pname='SSM1', 
                              )
    
    # output control
    oc = fp.mf6.ModflowGweoc(model=gwe,
                             saverecord=[("TEMPERATURE", "ALL"), ("BUDGET", "ALL")], # what to save
                             budget_filerecord=f"{gwename}.cbc", # file name where all budget output is stored
                             temperature_filerecord=f"{gwename}.ucn", # file name where all temperature output is stored
                            )
    
    fp.mf6.ModflowGwfgwe(simulation=sim, 
                         exgtype="GWF6-GWE6", 
                         exgmnamea=gwf.name, # name of groundwater flow model 
                         exgmnameb=gwe.name, # name of transport model
                         filename=f"{modelname}.gwfgwe",
                        );
    
    sim.write_simulation(silent=True)
    success, _ = sim.run_simulation(silent=True) 
    if success == 1:
        print('Model solved successfully')
    else:
        print('Solve failed')

    tempobj = gwe.output.temperature() # get handle to binary temperature file
    temp = tempobj.get_alldata().squeeze() # get the temperature data from the file
    times = np.array(tempobj.get_times()) # get the times and convert to array

    return temp, times, gwf, gwe

Running the model takes a bit longer than the ATES models in the previous notebooks (~10 seconds).

In [ ]:
# run model
ncycle = 3
temp, times, gwf, gwe = atesmodel(xwarm, ywarm, xcold, ycold, ncycle)

For illustration purposes, the head is plotted along the center row of the model.

In [ ]:
hds = gwf.output.head() # get handle to binary head file
head = hds.get_alldata().squeeze() # get all the head data from the file
times = np.array(hds.get_times()) # get times and make it an array

xg = gwf.modelgrid.xcellcenters[0] # row 0 of x-coordinates of cell centers of grid
plt.plot(xg, head[100, 30], label='warm injection')
plt.plot(xg, head[200, 30], label='cold injection')
plt.axvline(xwarm, color='C1', linestyle='--')
plt.axvline(xcold, color='C0', linestyle='--')
plt.title('head along center row of model')
plt.xlabel('x (m)')
plt.ylabel('head (m)')
plt.legend()
plt.grid()

### Read temperature data and make plot
The temperature data is read and plotted along the center row of the model both during injection of warm water and during injection of cold water. 

In [ ]:
def plot(t):
    tindex = int(t / delt) - 1
    plt.subplot(111, ylim=(Tcold - 0.2, Twarm + 0.2), xlabel='x (m)', ylabel='Temperature (\u00b0C)')
    plt.title('temperature along center row of model')
    plt.plot(xg, temp[tindex, 30])
    plt.axvline(xwarm, color='C1', linestyle='--')
    plt.axvline(xcold, color='C0', linestyle='--')
    plt.grid()

interact(plot, t=FloatSlider(value=delt, min=delt, max=ncycle * (tin + tout), 
                                description='time (days):', readout_format='.1f',
                                step=delt, layout=Layout(width='80%')));

A map-view plot of the temperature shows the growing and shrinking of the warm water and cold water bubbles. 

In [ ]:
def contour(t):
    plt.figure(figsize=(10, 5))
    tindex = int(t / delt) - 1
    pmv = fp.plot.PlotMapView(model=gwe)
    pa = pmv.plot_array(temp[tindex], vmin=7, vmax=17, cmap='Spectral_r') # plot an array by colored grid blocks
    cb = plt.colorbar(pa, shrink=0.9)
    cb.set_label("temperature (\u00b0C)")
    plt.plot([xwarm, xcold], [ywarm, ycold], 'k.')

interact(contour, t=FloatSlider(value=delt, min=delt, max=ncycle * (tin + tout), 
                                description='time (days):', readout_format='.1f',
                                step=delt, layout=Layout(width='70%')));

The temperature in the warm well and cold well are plotted below.

In [ ]:
iwarm, jwarm = gwf.modelgrid.intersect(xwarm, ywarm)
icold, jcold = gwf.modelgrid.intersect(xcold, ycold)
plt.plot(times, temp[:, iwarm, jwarm], 'C1', label='warm well')
plt.plot(times, temp[:, icold, jcold], 'C0', label='cold well')
plt.xlabel('Time (d)')
plt.ylabel('Temperature (\u00b0C)')
plt.xticks(np.arange(0, ncycle * (tin + tout) + 1, tin))
plt.ylim(Tcold - 0.5, Twarm + 0.5)
plt.yticks(np.linspace(Tcold, Twarm, 5))
plt.legend()
plt.grid()

Compute thermal recovery efficiency of the warm well for each cycle. 

In [ ]:
for icycle in range(ncycle):
    istart = nstepin + icycle * (nstepin + nstepout)
    iend = istart + nstepout
    energy_inj = Q * (Twarm - Tinit) * cpw * rhow * tin
    energy_ext = np.sum(Q * (temp[istart: iend, iwarm, jwarm] - Tinit) * cpw * rhow * delt)
    therm_rec_eff = energy_ext / energy_inj * 100
    print(f'cycle {icycle}, thermal recovery efficiency: {therm_rec_eff:.1f}%')

### Exercise
The warm and cold well are placed 200 m apart in the example above. When the wells are moved closer together, it is expected that the thermal recovery efficiency will go down. How close do you have to place them before the recovery efficiency of the warm well drops below 80% on the first cycle?

### Final comments
The model we created in this notebook is purposely more approximate than you may want in practice, so that the notebook runs fairly fast in the workshop. In practice, you have to use smaller cells, especially near the wells, and add semi-confining layers above and below the aquifer and divide the aquifer and semi-confining layers up in multiple model layers, as we did in the second notebook. This will significantly reduce the recovery efficiency (and increase the runtime). 